In [27]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import os

ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [13]:
df = pd.read_csv('shahname.csv')
df.head()

,Chapter,Part,Bait,Mesra,Text
0,1,1,1,1,به نام خداوند جان و خرد
1,1,1,1,2,کز این برتر اندیشه بر نگذرد
2,1,1,2,1,خداوند نام و خداوند جای
3,1,1,2,2,خداوند روزی ده رهنمای
4,1,1,3,1,خداوند کیوان و گَردان سپهر


In [14]:
text = '\n'.join(df['Text'])
print(text[:250])

به نام خداوند جان و خرد
کز این برتر اندیشه بر نگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خداوند کیوان و گَردان سپهر
فروزندهٔ ماه و ناهید و مهر
ز نام و نشان و گمان برتر است
نگارندهٔ بر شده پیکر است
به بینندگان آفریننده را
نبینی مرنجان دو بینن


In [16]:
tokens = text.split()
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['به', 'نام', 'خداوند', 'جان', 'و', 'خرد', 'کز', 'این', 'برتر', 'اندیشه', 'بر', 'نگذرد', 'خداوند', 'نام', 'و', 'خداوند', 'جای', 'خداوند', 'روزی', 'ده', 'رهنمای', 'خداوند', 'کیوان', 'و', 'گَردان', 'سپهر', 'فروزندهٔ', 'ماه', 'و', 'ناهید', 'و', 'مهر', 'ز', 'نام', 'و', 'نشان', 'و', 'گمان', 'برتر', 'است', 'نگارندهٔ', 'بر', 'شده', 'پیکر', 'است', 'به', 'بینندگان', 'آفریننده', 'را', 'نبینی', 'مرنجان', 'دو', 'بیننده', 'را', 'نیابد', 'بدو', 'نیز', 'اندیشه', 'راه', 'که', 'او', 'برتر', 'از', 'نام', 'و', 'از', 'جایگاه', 'سخن', 'هر', 'چه', 'زین', 'گوهران', 'بگذرد', 'نیابد', 'بدو', 'راه', 'جان', 'و', 'خرد', 'خرد', 'گر', 'سخن', 'برگزیند', 'همی', 'همان', 'را', 'گزیند،', 'که', 'بیند', 'همی', 'ستودن', 'نداند', 'کس،', 'او', 'را', 'چو', 'هست', 'میان', 'بندگی', 'را', 'ببایدت', 'بست', 'خرد', 'را', 'و', 'جان', 'را', 'همی', 'سنجد', 'اوی', 'در', 'اندیشهٔ', 'سخته', 'کی', 'گنجد', 'اوی', 'بدین', 'آلت', 'رای', 'و', 'جان', 'و', 'زبان', 'ستود', 'آفریننده', 'را', 'کی', 'توان', 'به', 'هستیش', 'باید', 'که', 'خستو', 'شوی

In [17]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 567122


In [21]:
print(sequences[:10])

['به نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خداوند کیوان و گَردان سپهر فروزندهٔ ماه و ناهید و مهر ز نام و نشان و گمان برتر است نگارندهٔ بر شده پیکر است به بینندگان آفریننده را نبینی مرنجان', 'نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خداوند کیوان و گَردان سپهر فروزندهٔ ماه و ناهید و مهر ز نام و نشان و گمان برتر است نگارندهٔ بر شده پیکر است به بینندگان آفریننده را نبینی مرنجان دو', 'خداوند جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خداوند کیوان و گَردان سپهر فروزندهٔ ماه و ناهید و مهر ز نام و نشان و گمان برتر است نگارندهٔ بر شده پیکر است به بینندگان آفریننده را نبینی مرنجان دو بیننده', 'جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خداوند کیوان و گَردان سپهر فروزندهٔ ماه و ناهید و مهر ز نام و نشان و گمان برتر است نگارندهٔ بر شده پیکر است به بینندگان آفریننده را نبینی مرنجان دو بیننده را', 'و خرد کز این

In [26]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(sequences)
inp_sequences[:10]

NameError: name 'Tokenizer' is not defined